# pmids.txt to pmids_df

In [1]:
import pandas as pd

pmids_df = pd.DataFrame({'PMID':[]})

In [2]:
pmids_df.head()

,PMID


In [3]:
with open('pmids.txt', 'r') as f:
    lines = f.readlines()
    lines = filter(lambda x: x and x.strip(), lines) # remove empty entities
    lines = map(lambda x: x.strip(), lines)
    lines = list(set(lines))# remove repeated entities
    pmids_df = pd.concat([pmids_df, pd.DataFrame({'PMID':list(lines)})])
    
    


In [4]:
pmids_df.head()

,PMID
0,25999454
1,23640184
2,18466097
3,24474498
4,26710337


# pmcids.txt to pmids_df

In [5]:
import requests
import time
import json

# should return 14585767 given 2869090
def pmcid2pmid(pmcid):
    url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pmc_pubmed&'+\
          f'id={pmcid}'+'&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08'
    try:
        rs = requests.get(url)
        time.sleep(0.1)
        rs_dict = json.loads(rs.text)
        item = rs_dict['linksets'][0]
        if 'linksetdbs' in item:
            return str(item['linksetdbs'][0]['links'][0])
        else:
            return None
    
    except requests.exceptions.RequestException as e:
        print(f'requests.get({url}) results in this error:')
        print(e)
    

In [33]:
pmcid2pmid('2869090')

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pmc_pubmed&id=2869090&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08


'14585767'

In [6]:
with open('pmcids.txt', 'r') as f:
    lines = f.readlines()
    lines = filter(lambda x: x and x.strip(), lines) # remove empty entities
    lines = map(lambda x: x.strip(), lines)
    lines = list(set(lines))# remove repeated entities
    lines = map(pmcid2pmid, lines)
    lines = filter(lambda x: x and x.strip(), lines) # remove entities
                                                     # that do not have corresponding pmids
    pmids_df = pd.concat([pmids_df, pd.DataFrame({'PMID':list(lines)})])

# dois.txt to pmids_df

In [12]:
# should return 20466091 given 10.1016/j.ajhg.2010.04.006
def doi2pmid(doi):
    
    url = 'https://www.ncbi.nlm.nih.gov/pmc/utils/idconv/v1.0/?'+\
          f'ids={doi}' + '&format=json&api_key=b9056d083d23b60058ea9dc7f9a802b2db08'
    try:
        rs = requests.get(url)
        time.sleep(0.1)
        rs_dict = json.loads(rs.text)
        item = rs_dict['records'][0]
        if 'pmid' in item:
            return item['pmid']
        else:
            return None
    
    except requests.exceptions.RequestException as e:
        print(f'requests.get({url}) results in this error:')
        print(e)

In [13]:
doi2pmid('10.1016/j.ajhg.2010.04.006')

'20466091'

In [14]:
with open('dois.txt', 'r') as f:
    lines = f.readlines()
    lines = filter(lambda x: x and x.strip(), lines) # remove empty entities
    lines = map(lambda x: x.strip(), lines)
    lines = list(set(lines))# remove repeated entities
    lines = map(doi2pmid, lines)
    lines = filter(lambda x: x and x.strip(), lines) # remove entities
                                                     # that do not have corresponding pmids
    pmids_df = pd.concat([pmids_df, pd.DataFrame({'PMID':list(lines)})])

In [15]:
pmids_df.count()

PMID    1411
dtype: int64

In [16]:
pmids_df.dtypes

PMID    object
dtype: object

In [18]:
pmids_df.drop_duplicates(keep='first', subset='PMID').count()

PMID    917
dtype: int64

In [19]:
pmids_df.to_csv('pmids.csv', index=False)